In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import rnn
import train

In [2]:
embedding_size = 10
hidden_size = 35
output_size = len(rnn.char.keys())
n_layers = 2
batch_size = 16
GAMMA = 0.99 # for exponential moving avarage constant from paper




In [3]:
print(rnn.token_num)

9


In [4]:
model = rnn.BFgen(rnn.token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [5]:
print(train.objective_PG(model, lambda x : np.array([1] * model.batch_size), N=16), model.entropy)

(Variable containing:
-1.8440
[torch.FloatTensor of size 1]
, Variable containing:
 3313.7212
[torch.FloatTensor of size 1]
)


In [6]:
print(train.objective_PQT(model))

Variable containing:
-184.4113
[torch.FloatTensor of size 1]



In [7]:
print(model.sample())

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [8]:
bfAdd = ",>++++++[<-------->-],[<+>-]<."
bfAdd2 = ",>++++++[<-------->-],[<+>-]<+."
bfAdd3 = ",>++++++[<-------->-],[<+>-]<++."
bfAdd4 = ",>++++++[<-------->-],[<+>-]<+++."
addBatch = [bfAdd, bfAdd2, bfAdd3, bfAdd4]
bfAddIn = ['11', '12', '23']
bfAddOut = ['2','3','5']

In [9]:
reward = train.batch_reward(bfAddIn, bfAddOut, len(addBatch))

In [10]:
reward(addBatch)

array([-76.8, -77.1, -77.1, -77.1])

In [11]:
reverse_in = ["a", "ab", "abc", "abcd", "abcde"]
reverse_out = ["a", "ba", "cba", "dcba", "edcba"]
reverse_reward = train.batch_reward(reverse_in, reverse_out, model.batch_size)

In [12]:
train.train_pqt_pg(model, reverse_reward)

Epoch Obj  Sample
----- ----- ------
   0  -7099.729 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
   1  -5140.440 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


KeyboardInterrupt: 